In [1]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

import wrangle, explore

import warnings
warnings.filterwarnings('ignore')

# Predicting Car Prices

In [2]:
#Talk about project scope

#### To-Do List
 - Create README on github that allows for project understanding
 - Find a dataset for used cars and there pricing 
 - create a function to put it into a pandas a dataframe
 - wrangle and explore, create a model that allows us to predict car price.
 - deploy this model into a phone app that allows you to  number 

## Wrangle 
 - The data We're working with here can be found on Kaggle
 - https://www.kaggle.com/ananaymital/us-used-cars-dataset

In [3]:
all_cars_df = pd.read_csv('used_cars_data.csv')

#### Summarize Statistics

In [4]:
all_cars_df.head()

,vin,back_legroom,bed,bed_height,bed_length,body_type,cabin,city,city_fuel_economy,combine_fuel_economy,...,transmission,transmission_display,trimId,trim_name,vehicle_damage_category,wheel_system,wheel_system_display,wheelbase,width,year
0,ZACNJABB5KPJ92081,35.1 in,NaN,NaN,NaN,SUV / Crossover,NaN,Bayamon,NaN,NaN,...,A,9-Speed Automatic Overdrive,t83804,Latitude FWD,NaN,FWD,Front-Wheel Drive,101.2 in,79.6 in,2019
1,SALCJ2FX1LH858117,38.1 in,NaN,NaN,NaN,SUV / Crossover,NaN,San Juan,NaN,NaN,...,A,9-Speed Automatic Overdrive,t86759,S AWD,NaN,AWD,All-Wheel Drive,107.9 in,85.6 in,2020
2,JF1VA2M67G9829723,35.4 in,NaN,NaN,NaN,Sedan,NaN,Guaynabo,17.0,NaN,...,M,6-Speed Manual,t58994,Base,NaN,AWD,All-Wheel Drive,104.3 in,78.9 in,2016
3,SALRR2RV0L2433391,37.6 in,NaN,NaN,NaN,SUV / Crossover,NaN,San Juan,NaN,NaN,...,A,8-Speed Automatic Overdrive,t86074,V6 HSE AWD,NaN,AWD,All-Wheel Drive,115 in,87.4 in,2020
4,SALCJ2FXXLH862327,38.1 in,NaN,NaN,NaN,SUV / Crossover,NaN,San Juan,NaN,NaN,...,A,9-Speed Automatic Overdrive,t86759,S AWD,NaN,AWD,All-Wheel Drive,107.9 in,85.6 in,2020


In [41]:
#dataset is huge , we'll grab 125,000 observations
cars_df = all_cars_df.sample(50000)

In [42]:
#wrangle.summerize_df(cars_df)

#### Null Handling

In [43]:
col = wrangle.get_nulls(cars_df)[0]

<class 'pandas.core.series.Series'>


In [44]:
col[1:61]

,num_rows_missing,percent_rows_missing
back_legroom,2685,5.370
bed,49670,99.340
bed_height,42823,85.646
bed_length,42823,85.646
body_type,207,0.414
cabin,48924,97.848
city,0,0.000
city_fuel_economy,8137,16.274
combine_fuel_economy,50000,100.000
daysonmarket,0,0.000


In [45]:
col[61:]

,num_rows_missing,percent_rows_missing
wheel_system,2491,4.982
wheel_system_display,2491,4.982
wheelbase,2685,5.370
width,2685,5.370
year,0,0.000


In [46]:
cars_df.shape

(50000, 66)

#### High Null Columns takeaways (Over 90%)

- Bed is only used for trucks, making the column 99% null , remove bed info
- Cabin missing 97%, remove cabin
- Combine Fuel Economy is 100% null, remove
- drop is_certified
- drop is_cpo

In [47]:
cars_df = wrangle.drop_null_columns(cars_df , 90)

<class 'pandas.core.series.Series'>


In [48]:
cars_df.shape

(50000, 59)

#### High Null rows takeaways (Over 40%)
- removing any row that has more than 40% of its columns missing

In [49]:
row = wrangle.get_nulls(cars_df)[1]

<class 'pandas.core.series.Series'>


In [50]:
row

,num_cols_missing,percent_cols_missing,num_rows
0,0,0.0,1112
1,1,1.694915254237288,819
2,2,3.389830508474576,9887
3,3,5.084745762711865,7631
4,4,6.779661016949152,3099
5,5,8.47457627118644,1584
6,6,10.16949152542373,510
7,7,11.864406779661017,2216
8,8,13.559322033898304,713
9,9,15.254237288135593,10351


In [51]:
cars_df = wrangle.drop_null_rows(cars_df, 40)

In [52]:
cars_df.shape

(48382, 59)

In [53]:
cars_df.isna().sum()

vin                         0
back_legroom             1067
bed_height              41205
bed_length              41205
body_type                  19
city                        0
city_fuel_economy        6547
daysonmarket                0
dealer_zip                  0
description              1200
engine_cylinders          818
engine_displacement      1320
engine_type               818
exterior_color              0
fleet                   22333
frame_damaged           22333
franchise_dealer            0
franchise_make           9357
front_legroom            1067
fuel_tank_volume         1067
fuel_type                 626
has_accidents           22333
height                   1067
highway_fuel_economy     6547
horsepower               1320
interior_color              0
isCab                   22333
is_new                      0
latitude                    0
length                   1067
listed_date                 0
listing_color               0
listing_id                  0
longitude 

#### Investigating remaining nulls (To drop, fill with a value, or impute an avg)

In [54]:
#Bed
#Bed stats only used for pickuptruck, remove
cars_df = cars_df.drop(columns=['bed_height', 'bed_length'])

In [55]:
cars_df.shape

(48382, 57)

In [57]:
#Body type is important and card to impute,  drop the missing rows
cars_df = cars_df.dropna(axis=0, subset=['body_type'])

In [58]:
#back_legroom , lets take a closer look into the null values
cars_df['back_legroom'].value_counts()


38.3 in    1913
43.6 in    1771
37.4 in    1429
--         1363
35.7 in    1265
           ... 
13 in         1
28.8 in       1
45.1 in       1
23.7 in       1
24 in         1
Name: back_legroom, Length: 179, dtype: int64

In [59]:
#leg_room
split = cars_df['back_legroom'].str.split(' ', n =1, expand = True)
split1 = cars_df['front_legroom'].str.split(' ', n =1, expand = True)

In [60]:
cars_df['back_legroom'] = split[0]
cars_df['front_legroom'] = split1[0]

In [61]:
cars_df['back_legroom'] = pd.to_numeric(cars_df['back_legroom'], errors='coerce')
cars_df['front_legroom'] = pd.to_numeric(cars_df['front_legroom'], errors='coerce')

In [62]:
cars_df.groupby(by='body_type').back_legroom.mean()

body_type
Convertible        30.325581
Coupe              31.850107
Hatchback          34.795807
Minivan            38.512942
Pickup Truck       40.086644
SUV / Crossover    38.339480
Sedan              37.053246
Van                35.730939
Wagon              37.279391
Name: back_legroom, dtype: float64

In [63]:
#fill nulls with avg legroom for each body type
cars_df['back_legroom'] = cars_df.groupby('body_type').transform(lambda x : x.fillna(round(x.mean(),1)))
cars_df['front_legroom'] = cars_df.groupby('body_type').transform(lambda x : x.fillna(round(x.mean(),1)))

In [64]:
#city_fuel_economy can be filled with the avg for year, make and model
cars_df['city_fuel_economy'] = cars_df.groupby(['year','make_name','model_name']).transform(lambda x : x.fillna(round(x.mean(),1)))
#highway_fuel_economy as well
cars_df['highway_fuel_economy'] = cars_df.groupby(['year','make_name','model_name']).transform(lambda x : x.fillna(round(x.mean(),1)))

In [65]:
#drop description, cant be used in regression model
cars_df = cars_df.drop(columns='description')

In [66]:
#engine specifics 
#use the year, make and model to figure out what engine might have been in that particular vehicle

In [67]:
#fill engine specifics with most popular option for that year , make and model
cars_df['engine_cylinders'] = cars_df.groupby(['year','make_name','model_name']).\
transform(lambda x : x.fillna(x.mode()))
cars_df['engine_displacement'] = cars_df.groupby(['year','make_name','model_name']).\
transform(lambda x : x.fillna(x.mode()))
cars_df['engine_type'] = cars_df.groupby(['year','make_name','model_name']).\
transform(lambda x : x.fillna(x.mode()))

In [68]:
#fleet can be filled with "unknown" as being a fleet car might have an effect on its pricing
#but we dont want to encode to binary, as it may skew our non fleet cars
cars_df.fleet = cars_df.fleet.fillna('unknown')

In [75]:
#Assume seller verified no frame damage, impute nulls to false
cars_df.frame_damaged = cars_df.frame_damaged.fillna(False)

In [79]:
#franchise make is not important, drop columns
cars_df = cars_df.drop(columns='franchise_make')

In [80]:
#fuel type specifics can be imputed by engine type
cars_df['fuel_type'] = cars_df.groupby(['engine_type']).\
transform(lambda x : x.fillna(x.mode()))